In [5]:
#!/opt/anaconda/envs/bd9/bin/python
import happybase
import sys

In [6]:
import fileinput
import os
from decimal import Decimal
import re

### Let's check the properties of our the table

In [1]:
#connection = happybase.Connection('bd-node2.newprolab.com')
#connection.open()
#connection.delete_table('valeria.lupanova', disable=True)
#connection.close()

In [2]:
#families={'facetz_2015_02_12':dict()}

In [3]:
#connection = happybase.Connection('bd-node2.newprolab.com')
#connection.open()
#connection.create_table(name='valeria.lupanova', families=families)
#connection.close()

In [7]:
connection = happybase.Connection('bd-node2.newprolab.com')
connection.open()
#print(connection.tables())
table = connection.table('valeria.lupanova')
print(table.families())
print(table.regions())
connection.close()

{b'data': {'name': b'data:', 'max_versions': 4096, 'compression': b'NONE', 'in_memory': False, 'bloom_filter_type': b'ROW', 'bloom_filter_vector_size': 0, 'bloom_filter_nb_hashes': 0, 'block_cache_enabled': True, 'time_to_live': 2147483647}}
[{'start_key': b'', 'end_key': b'', 'id': 1602885030676, 'name': b'valeria.lupanova,,1602885030676.9b7761610f689cdde2defbfed514fcf8.', 'version': 1, 'server_name': b'bd-node4.newprolab.com', 'port': 16020}]


### Here I try to emit one row of data to the table

In [30]:
#connection = happybase.Connection('bd-node2.newprolab.com')
#connection.open()
#table = connection.table('valeria.lupanova')
#table.put(row=b'413', data={b'data:url':b'http%3A%2F%2Fwww.ridus.ru%2F'}, timestamp=1423684799735)
#connection.close()

### Here I try to read one file, map it, preprocess and look at the result

In [ ]:
#os.listdir('facetz_2015_02_12')

In [54]:
for file in os.listdir('facetz_2015_02_12'):
    data_path = "facetz_2015_02_12"
    #file_name = "part-00000"
    with open(os.path.join(data_path, file), "r") as f:
        content = f.read()
    content_lines = content.splitlines()
    print(file)
    for line in content_lines:
        mapper(line)

part-00068
part-00171
part-00168
part-00021
part-00146
part-00050
part-00023
part-00092
part-00154
part-00086
part-00089
part-00156
part-00137
part-00148
part-00142
part-00051
part-00090
part-00017
part-00083
part-00120
part-00070
part-00003
1671507613 1423692352353 http%3A//www.as8.ru/forum/
Success!
part-00005
part-00001
part-00006
part-00088
part-00054
part-00058
part-00094
part-00080
part-00134
part-00019
part-00057
part-00150
part-00045
part-00066
part-00053
part-00047
part-00166
part-00119
part-00069
part-00007
part-00015
part-00048
part-00101
part-00122
part-00174
part-00115
part-00097
part-00013
part-00157
part-00112
part-00132
part-00099
part-00010
part-00091
part-00110
part-00008
part-00133
part-00103
part-00022
part-00118
part-00173
part-00114
part-00063
part-00106
part-00000
part-00107
part-00076
part-00079
part-00129
part-00038
part-00130
part-00169
part-00073
part-00087
part-00136
part-00042
part-00084
part-00043
part-00027
part-00144
part-00140
part-00039
part-00034
part

In [46]:
#data_path = "facetz_2015_02_12"
#file_name = "part-00000"
#with open(os.path.join(data_path, file_name), "r") as f:
#    content = f.read()
#content_lines = content.splitlines()
#for line in content_lines[:10]:
#    mapper(line)

In [8]:
def mapper(line):
    uid, ts, url = line.split('\t')
    if re.match(r'^[\d]+$', (uid.split('\t'))[0]):
        if re.match(r'^[\d]+\.[\d]+$', ts):
            if re.match(r'^http.+$', url):
                if int(uid)%256==157:
                    ts = int(float(ts)*1000)
                    print(uid, ts, url)
                    emitter(uid, ts, url)

In [9]:
def emitter(uid, ts, url):
    connection = happybase.Connection('bd-node2.newprolab.com')
    connection.open()
    table = connection.table('valeria.lupanova')
    try:
        table.put(row=str(uid), data={b'data:url':url}, timestamp=ts)
    except:
        print('Something went wrong!')
    else:
        print('Success!')
    finally:
        connection.close()